In [131]:
import pandas as pd

data: str = '../../data/raw/wordle_withclue_old_raw.jsonl'
save_path: str = '../../data/processed/wordle_withclue_old_processed.jsonl'
df = pd.read_json(data, lines=True)

# This works for wordle and wordle with clue

## Filter all conversations that are successful and have an invalid format inside the assistant text

In [132]:
# Define the text to filter out
specific_text = 'INVALID_FORMAT'

# Function to check if specific_text is in any of the dictionaries in the list
def contains_specific_text(row):
      return any(specific_text in d['content'] for d in row)

In [133]:
# Filter the DataFrame
filtered_data = df[~df['chat'].apply(contains_specific_text)]

In [134]:
len(df), len(filtered_data)

(1497, 1376)

# Clean Up all episodes with repeated prompting

In [135]:
def filter_repeated_interaction(chat: list) -> list:
      cleaned_chat = []
      skip_next = False
      for turn in chat:
            if skip_next:
                  skip_next = False
                  continue
            try:
                  if turn['has_error']:
                        skip_next = True
                  else:
                     cleaned_chat.append({
                           'role': turn['role'],
                           'content': turn['content'],
                     })
            except KeyError:
                  cleaned_chat.append(
                        {
                              'role': turn['role'],
                              'content': turn['content'],
                        }
                  )
                      
      return cleaned_chat

len(filtered_data.chat.iloc[0]), len(filter_repeated_interaction(filtered_data.chat.iloc[0]))

(12, 12)

In [136]:
filtered_data.chat = filtered_data.chat.apply(filter_repeated_interaction)

/var/folders/qh/qhq087rn6n9bnym2l00z7qyw0000gn/T/ipykernel_74909/3763763122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.chat = filtered_data.chat.apply(filter_repeated_interaction)


## Finding:
Wordle With clue and critic has a lot of INVALID_FORMAT in its explanations. When this is used to train the model to deliver content, 
it will learn to produce only the INVALID_CONTENT token. This does not help the game to be better played

In [137]:
filtered_data = filtered_data.rename(columns={'target_word': 'target'})
filtered_data = filtered_data.drop(['target_word_difficulty', 'target_word_clue'], axis=1)
filtered_data['player'] = "Player 1"

In [138]:
filtered_data.to_json(save_path, orient='records', lines=True)

In [139]:
filtered_data.head()

,game,game_id,model,benchmark_version,experiment,episode,Aborted,Lose,Success,chat,target,player
0,wordle_withclue,1,claude-v1.3-t0.0--claude-v1.3-t0.0,v0.9,0_high_frequency_words_clue_no_critic,episode_0,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",acute,Player 1
1,wordle_withclue,2,claude-v1.3-t0.0--claude-v1.3-t0.0,v0.9,0_high_frequency_words_clue_no_critic,episode_1,0,0,1,"[{'role': 'user', 'content': 'You are a langua...",blade,Player 1
2,wordle_withclue,3,claude-v1.3-t0.0--claude-v1.3-t0.0,v0.9,0_high_frequency_words_clue_no_critic,episode_2,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",wacky,Player 1
3,wordle_withclue,4,claude-v1.3-t0.0--claude-v1.3-t0.0,v0.9,0_high_frequency_words_clue_no_critic,episode_3,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",extol,Player 1
4,wordle_withclue,5,claude-v1.3-t0.0--claude-v1.3-t0.0,v0.9,0_high_frequency_words_clue_no_critic,episode_4,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",polka,Player 1


In [121]:
df.head()

,game,game_id,model,benchmark_version,experiment,episode,Aborted,Lose,Success,chat,target_word,target_word_difficulty,target_word_clue
0,wordle,1,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_no_clue_no_critic,episode_0,1,0,0,"[{'role': 'user', 'content': 'You are a langua...",start,high_frequency,embark
1,wordle,2,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_no_clue_no_critic,episode_1,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",share,high_frequency,lion's amount?
2,wordle,3,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_no_clue_no_critic,episode_2,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",third,high_frequency,bronze place
3,wordle,4,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_no_clue_no_critic,episode_3,1,0,0,"[{'role': 'user', 'content': 'You are a langua...",photo,high_frequency,pic
4,wordle,5,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_no_clue_no_critic,episode_4,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",might,high_frequency,strength
